# MegaDescriptor-B-224
- Configuration used for training MegaDescriptor-B-224 (https://huggingface.co/BVRA/MegaDescriptor-B-224)

In [ ]:
from itertools import chain
import torch
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD
from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer


# Dataset configuration
metadata = pd.read_csv('../data/metadata/combined/combined_all.csv')
image_root = '../data/images/size-256'
transform = T.Compose([
    T.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    T.RandAugment(num_ops=2, magnitude=20),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])
dataset = WildlifeDataset(
    metadata = metadata, 
    root = image_root,
    split = SplitMetadata('split', 'train'),
    transform=transform
)


# Backbone and loss configuration
backbone = timm.create_model('swin_base_patch4_window7_224', num_classes=0, pretrained=True)
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 224, 224)
    embedding_size = backbone(dummy_input).shape[1]
objective = ArcFaceLoss(num_classes=dataset.num_classes, embedding_size=embedding_size, margin=0.5, scale=64)


# Optimizer and scheduler configuration
params = chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)
min_lr = optimizer.defaults.get("lr") * 1e-3
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=min_lr)


# Setup training
trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=64,
    accumulation_steps=2,
    num_workers=2,
    epochs=100,
    device='cuda',
)


trainer.train()